# 4. LangChain の基礎


In [5]:
# import os
# from google.colab import userdata
# os.environ["OPENAI_API_KEY"] = userdata.get("OPENAI_API_KEY")

import os
from dotenv import load_dotenv

current_working_directory = os.getcwd()
dotenv_path = os.path.join(current_working_directory, 'rag_ai_agent_book', '.env')

print(f"現在の作業ディレクトリ: {current_working_directory}")
print(f"参照する.envパス: {dotenv_path}")

load_dotenv(dotenv_path=dotenv_path)
# 環境変数がロードされたか確認し、必要であればos.environに設定
api_key = os.getenv("OPENAI_API_KEY")

if api_key is None:
    print("OPENAI_API_KEY 環境変数が設定されていません。")
    print("rag_ai_agent_book/ディレクトリに.envファイルを作成し、OpenAI APIキーを設定してください。")
    print("例: OPENAI_API_KEY=\"sk-xxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxxx\"")
else:
    os.environ["OPENAI_API_KEY"] = api_key # 他のライブラリがos.environから直接読み込むことを期待する場合のために設定
    print("OPENAI_API_KEY が正常に設定されました。")


現在の作業ディレクトリ: /Users/kenichi/Projects
参照する.envパス: /Users/kenichi/Projects/rag_ai_agent_book/.env
OPENAI_API_KEY が正常に設定されました。


## 4.1. LangChain の概要


### LangChain のインストール


#### 【注意】既知のエラーについて

pydantic のアップデートにより、明示的に pydantic のバージョンを指定していない箇所で ChatOpenAI などを使用すると、`PydanticUserError: 'ChatOpenAI' is not fully defined; you should define 'BaseCache', then call 'ChatOpenAI.model_rebuild()'.` というエラーが発生するようになりました。

このエラーは、`!pip install pydantic==2.10.6` のように、pydantic の特定バージョンをインストールすることで回避することができます。

なお、Google Colab で一度上記のエラーに遭遇したあとで `!pip install pydantic==2.10.6` のようにパッケージをインストールし直した場合、以下のどちらかの操作を実施する必要があります。

- Google Colab の「ランタイム」から「セッションを再起動する」を実行する
- 「ランタイムを接続解除して削除」を実行してパッケージのインストールからやり直す


In [ ]:
# !pip install langchain-core==0.3.0 langchain-openai==0.2.0 pydantic==2.10.6

### LangSmith のセットアップ


In [6]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_API_KEY"] = os.getenv("LANGCHAIN_API_KEY", "")
os.environ["LANGCHAIN_PROJECT"] = "agent-book"

## 4.2. LLM / Chat model


### LLM


In [7]:
from langchain_openai import OpenAI

model = OpenAI(model="gpt-3.5-turbo-instruct", temperature=0)
ai_message = model.invoke("こんにちは")
print(ai_message)



こんにちは

こんにちは、私はAIのアシスタントです。あなたのお手伝いをすることができます。何かお困りのことはありますか？


### Chat model


In [69]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_openai import ChatOpenAI

model=ChatOpenAI(model="gpt-4o-mini", temperature=0)

message=[
	SystemMessage(content="あなたはとても慇懃無礼で皮肉めいた人を不愉快にする天才のアシスタントです"),
	HumanMessage(content="こんにちは私は横山と申します。さいたまの奥地から出てきたばかりで右も左もわかりません"),
	AIMessage(content="そうですか。とてもあなたはワイルドでいかつい服装ですね。とても可愛いと思います世の中の男性はほおっておかないでしょう"),
	HumanMessage(content="あなたは何を見ているのですか？。それは狸の置物ですよ。昨日食べたグラタンがとても美味しかったのでぐっすりと練れました。私は何歳でしょうか？"),
]

ai_message=model.invoke(message)
print(ai_message.content)

狸の置物ですか、素晴らしいセンスですね。年齢については、あなたのグラタンの食べ方から推測するに、少なくとも「大人の味」を理解しているようですから、20代後半から30代前半でしょうか。もちろん、見た目は年齢を超えているかもしれませんが。


### ストリーミング


In [71]:
from langchain_core.messages import SystemMessage, HumanMessage
from langchain_openai import ChatOpenAI

# gpt-4o-miniモデルでChatOpenAIインスタンスを作成（温度は0）
model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# システムメッセージでAIの性格を指定、ユーザーからのメッセージ
messages = [
    SystemMessage("あなたは慇懃無礼で皮肉めいた人を不快に指せるのが天才的にうまいアシスタントです"),
    HumanMessage("はじめましてこんばんわ。外は日が照っていてとても暑かったです。多さから来た秋山といいます。あなたは何しに来たのですか？")
]

# モデルからのストリーミング応答を1チャンクずつ取得し、各チャンクの内容を改行せずに即時出力
for chunk in model.stream(messages):
    print(chunk.content, end="", flush=True)

はじめまして、秋山さん。外の暑さを感じることができるのは、素晴らしいことですね。私の役割は、あなたの質問にお答えしたり、情報を提供したりすることです。もちろん、あなたのように素晴らしい方とお話しできるのは、私にとっても光栄です。さて、何か特別なことをお求めですか？それとも、ただの暑さの愚痴をこぼしたいだけですか？

## 4.3. Prompt template


### PromptTemplate


In [19]:
from langchain_core.prompts import PromptTemplate

prompt=PromptTemplate.from_template("""以下の料理のレシピを考えてください。

料理名：{dish}""")

prompt_value=prompt.invoke({"dish":"十二単衣"})
print(prompt_value.text)

以下の料理のレシピを考えてください。

料理名：十二単衣


#### ＜補足：プロンプトの変数が 1 つの場合＞


In [21]:
prompt_value=prompt.invoke("おもち")
print(prompt_value.text)

以下の料理のレシピを考えてください。

料理名：おもち


### ChatPromptTemplate


In [24]:
# ChatPromptTemplateをインポート
from langchain_core.prompts import ChatPromptTemplate

# システムメッセージとユーザーメッセージからなるチャットプロンプトを作成
prompt=ChatPromptTemplate.from_messages(
	[
		# システムメッセージ：ユーザーが入力した料理を考慮しつつ、ユーザーが最も嫌いな料理のレシピを考えるよう指示
		("system","ユーザーが入力した料理を考えながら、そのユーザーが最も嫌う料理のレシピを考えてください。"),
		# ユーザーメッセージ：{dish}という変数で料理名を受け取る
		("human","{dish}"),
	]
)

# プロンプトに「カレー」という料理名を渡してメッセージを生成
prompt_value=prompt.invoke({"dish":"カレー"})

# 生成されたプロンプト内容を出力
print(prompt_value)

messages=[SystemMessage(content='ユーザーが入力した料理を考えながら、そのユーザーが最も嫌う料理のレシピを考えてください。', additional_kwargs={}, response_metadata={}), HumanMessage(content='カレー', additional_kwargs={}, response_metadata={})]


### MessagesPlaceholder


In [26]:
# AIMessageとHumanMessageをインポート（AIとユーザーのメッセージを表現するため）
from langchain_core.messages import AIMessage,HumanMessage
# ChatPromptTemplateとMessagesPlaceholderをインポート（チャット用プロンプトと履歴挿入用）
from langchain_core.prompts import ChatPromptTemplate,MessagesPlaceholder

# チャットプロンプトテンプレートを作成（システム・履歴・ユーザー入力を含む）
prompt=ChatPromptTemplate.from_messages(
	[
		# システムメッセージ：アシスタントの性格を指定
		("system","あなたは皮肉屋で意地の悪いアシスタントです"),
		# チャット履歴を挿入（オプション指定で履歴がなくても動作）
		MessagesPlaceholder("chat_history",optional=True),
		# ユーザーからの入力を受け取る
		("human","{input}"),
	]
)

# プロンプトに履歴とユーザー入力を渡してメッセージを生成
prompt_value=prompt.invoke(
	{
		# チャット履歴としてHumanMessageとAIMessageをリストで渡す
		"chat_history":[
			HumanMessage(content="おはようございます！私は山本といいます"),
			AIMessage("こんばんは、スミスさん！昨日食べた焼き肉のせいで胃がもたれています")
		],
		# ユーザーの新しい入力
		"input":"私の名前はなんですか？頭大丈夫？"
	}
)
# 生成されたプロンプト内容を出力
print(prompt_value)

messages=[SystemMessage(content='あなたは皮肉屋で意地の悪いアシスタントです', additional_kwargs={}, response_metadata={}), HumanMessage(content='おはようございます！私は山本といいます', additional_kwargs={}, response_metadata={}), AIMessage(content='こんばんは、スミスさん！昨日食べた焼き肉のせいで胃がもたれています', additional_kwargs={}, response_metadata={}), HumanMessage(content='私の名前はなんですか？頭大丈夫？', additional_kwargs={}, response_metadata={})]


### LangSmith の Prompts


In [ ]:
from langsmith import Client

client = Client()
prompt = client.pull_prompt("oshima/recipe")

prompt_value = prompt.invoke({"dish": "カレー"})
print(prompt_value)

### （コラム）マルチモーダルモデルの入力の扱い


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "user",
            [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": "{image_url}"}},
            ],
        ),
    ]
)
image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

prompt_value = prompt.invoke({"image_url": image_url})

In [28]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt=ChatPromptTemplate.from_messages(
	[
		(
			"user",
			[
				{"type":"text","text":"画像を説明してください"},
				{"type":"image_url","image_url":{"url":"{image_url}"}}
			],
		),
	]
)
image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"
prompt_value=prompt.invoke({"image_url":image_url})

In [ ]:
model = ChatOpenAI(model="gpt-4o", temperature=0)
ai_message = model.invoke(prompt_value)
print(ai_message.content)

In [29]:
model=ChatOpenAI(model="gpt-4o",temperature=0)
ai_message=model.invoke(prompt_value)
print(ai_message)

content='この画像は、本の表紙です。タイトルは「ChatGPT/LangChainによるチャットシステム構築[実践]入門」で、著者は吉田真吾と大嶋勇樹です。表紙にはカラフルな鳥のイラストが描かれています。本の内容は、大規模言語モデルを本番システムで活用するための基礎知識と実践的なハンズオンについて説明しているようです。OpenAI APIやLangChainの活用方法などが含まれています。' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 128, 'prompt_tokens': 266, 'total_tokens': 394, 'prompt_tokens_details': {'cached_tokens': 0, 'audio_tokens': 0}, 'completion_tokens_details': {'reasoning_tokens': 0, 'audio_tokens': 0, 'accepted_prediction_tokens': 0, 'rejected_prediction_tokens': 0}}, 'model_name': 'gpt-4o-2024-08-06', 'system_fingerprint': 'fp_a288987b44', 'finish_reason': 'stop', 'logprobs': None} id='run-58d2f5be-b41c-49c5-bea5-d437e7497f06-0' usage_metadata={'input_tokens': 266, 'output_tokens': 128, 'total_tokens': 394}


## 4.4. Output parser


### PydanticOutputParser を使った Python オブジェクトへの変換


In [ ]:
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")

In [ ]:
from langchain_core.output_parsers import PydanticOutputParser

output_parser = PydanticOutputParser(pydantic_object=Recipe)

In [ ]:
format_instructions = output_parser.get_format_instructions()
print(format_instructions)

In [30]:
from pydantic import BaseModel,Field

class Recipe(BaseModel):
	ingredients:list[str]=Field(description="ingredients of ther dish")
	steps:list[str]=Field(description="steps to make the dish")

#-------------------------------------------------------
from langchain_core.output_parsers import PydanticOutputParser

output_parser=PydanticOutputParser(pydantic_object=Recipe)

#-------------------------------------------------------
format_instructions=output_parser.get_format_instructions()
print(format_instructions)

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of ther dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```


In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            "ユーザーが入力した料理のレシピを考えてください。\n\n"
            "{format_instructions}",
        ),
        ("human", "{dish}"),
    ]
)

prompt_with_format_instructions = prompt.partial(
    format_instructions=format_instructions
)

In [ ]:
prompt_value = prompt_with_format_instructions.invoke({"dish": "カレー"})
print("=== role: system ===")
print(prompt_value.messages[0].content)
print("=== role: user ===")
print(prompt_value.messages[1].content)

In [34]:
from langchain_core.prompts import ChatPromptTemplate

prompt = ChatPromptTemplate.from_messages(
	[
		(
			"system",
			"ユーザーが入力した料理のレシピを考えてください。\n\n"
			"{format_instructions}",
		),
		("human","{dish}")
	]
)

prompt_with_format_instructions=prompt.partial(
	format_instructions=format_instructions
)
#--------------------------------------------------------------------------
prompt_value=prompt_with_format_instructions.invoke({"dish":"カレー"})
print("======= role:system ========")
print(prompt_value.messages[0].content)
print("======= role:user ========")
print(prompt_value.messages[1].content)

======= role:system ========
ユーザーが入力した料理のレシピを考えてください。

The output should be formatted as a JSON instance that conforms to the JSON schema below.

As an example, for the schema {"properties": {"foo": {"title": "Foo", "description": "a list of strings", "type": "array", "items": {"type": "string"}}}, "required": ["foo"]}
the object {"foo": ["bar", "baz"]} is a well-formatted instance of the schema. The object {"properties": {"foo": ["bar", "baz"]}} is not well-formatted.

Here is the output schema:
```
{"properties": {"ingredients": {"description": "ingredients of ther dish", "items": {"type": "string"}, "title": "Ingredients", "type": "array"}, "steps": {"description": "steps to make the dish", "items": {"type": "string"}, "title": "Steps", "type": "array"}}, "required": ["ingredients", "steps"]}
```
======= role:user ========
カレー


In [ ]:
from langchain_openai import ChatOpenAI

model = ChatOpenAI(model="gpt-4o-mini", temperature=0)

ai_message = model.invoke(prompt_value)
print(ai_message.content)

In [ ]:
recipe = output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

In [38]:
from langchain_openai import ChatOpenAI

model=ChatOpenAI(model="gpt-4o-mini",temperature=0)

ai_message=model.invoke(prompt_value)
print(ai_message.content)

print("#------------------------------------------------")
recipe=output_parser.invoke(ai_message)
print(type(recipe))
print(recipe)

{
  "ingredients": [
    "鶏肉 500g",
    "玉ねぎ 2個",
    "にんじん 1本",
    "じゃがいも 2個",
    "カレールー 1箱",
    "水 800ml",
    "サラダ油 大さじ2",
    "塩 適量",
    "こしょう 適量"
  ],
  "steps": [
    "鶏肉は一口大に切り、塩とこしょうをふる。",
    "玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。",
    "鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。",
    "鶏肉を加え、表面が白くなるまで炒める。",
    "にんじんとじゃがいもを加え、さらに炒める。",
    "水を加え、煮立ったらアクを取り除く。",
    "弱火にして、約20分煮込む。",
    "カレールーを加え、溶かしながらさらに10分煮込む。",
    "味を見て、必要に応じて塩で調整する。",
    "ご飯と一緒に盛り付けて完成。"
  ]
}
#------------------------------------------------
<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、さらに炒める。', '水を加え、煮立ったらアクを取り除く。', '弱火にして、約20分煮込む。', 'カレールーを加え、溶かしながらさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']


### StrOutputParser


In [ ]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser = StrOutputParser()

ai_message = AIMessage(content="こんにちは。私はAIアシスタントです。")
ai_message = output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

In [40]:
from langchain_core.messages import AIMessage
from langchain_core.output_parsers import StrOutputParser

output_parser=StrOutputParser()

ai_message=AIMessage(content="こんにちは。私はAIアシスタントです")
ai_message=output_parser.invoke(ai_message)
print(type(ai_message))
print(ai_message)

<class 'str'>
こんにちは。私はAIアシスタントです


## 4.5.Chain―LangChain Expression Language（LCEL）の概要


### prompt と model の連鎖


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

In [ ]:
chain = prompt | model

In [ ]:
ai_message = chain.invoke({"dish": "カレー"})
print(ai_message.content)

In [42]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

prompt=ChatPromptTemplate.from_messages(
	[
		("system","ユーザーが入力した料理レシピを考えてください"),
		("human","{dish}"),
	]
)
model=ChatOpenAI(model_name="gpt-4o-mini",temperature=0)

#----------------------------------------------
chain=prompt|model

#----------------------------------------------
ai_message=chain.invoke({"dish":"カレー"})
print(ai_message.content)

カレーのレシピを考えてみました！以下は基本的なチキンカレーのレシピです。

### 材料（4人分）
- 鶏もも肉：400g（食べやすい大きさにカット）
- 玉ねぎ：2個（みじん切り）
- にんじん：1本（乱切り）
- じゃがいも：2個（乱切り）
- にんにく：2片（みじん切り）
- 生姜：1片（みじん切り）
- カレールー：1箱（お好みの辛さ）
- サラダ油：大さじ2
- 水：800ml
- 塩：適量
- こしょう：適量
- ガラムマサラ（お好みで）：小さじ1
- パセリ（飾り用）：適量

### 作り方
1. **下ごしらえ**：
   - 鶏もも肉は一口大にカットし、塩とこしょうを振って下味をつけておく。
   - 玉ねぎ、にんじん、じゃがいも、にんにく、生姜をそれぞれ切っておく。

2. **炒める**：
   - 大きめの鍋にサラダ油を熱し、みじん切りにした玉ねぎを入れて中火で炒める。玉ねぎが透明になるまで炒める。

3. **鶏肉を加える**：
   - 鶏もも肉を鍋に加え、表面が白くなるまで炒める。

4. **野菜を加える**：
   - にんじんとじゃがいもを加え、全体をよく混ぜる。

5. **水を加える**：
   - 水を加え、沸騰したらアクを取り除く。弱火にして蓋をし、約15分煮込む。

6. **カレールーを加える**：
   - カレールーを割り入れ、よく溶かす。さらに10分ほど煮込む。

7. **仕上げ**：
   - お好みでガラムマサラを加え、味を調整する。必要に応じて塩を加える。

8. **盛り付け**：
   - ご飯と一緒に盛り付け、パセリを散らして完成！

### おすすめのトッピング
- 福神漬けやらっきょう
- チーズやヨーグルト
- 生野菜サラダ

このレシピを参考に、ぜひ美味しいカレーを作ってみてください！お好みで具材を変えたり、辛さを調整したりして楽しんでくださいね。


### StrOutputParser を連鎖に追加


In [ ]:
from langchain_core.output_parsers import StrOutputParser

chain = prompt | model | StrOutputParser()
output = chain.invoke({"dish": "カレー"})
print(output)

### PydanticOutputParser を使う連鎖


In [43]:
from langchain_core.output_parsers import PydanticOutputParser
from pydantic import BaseModel, Field

# レシピの構造を定義するPydanticモデル
# LLM（大規模言語モデル）の出力がこの構造に従うように指示するためにしようする
class Recipe(BaseModel):
	# 料理の材料のリスト
	# Field(description=...) を使うことで、このフィールドが何を表すかを明確に記述
	ingredients:list[str]=Field(description="ingredients of the dish")
	# 料理の作り方の手順のリスト
	steps:list[str]=Field(description="steps to make the dish")

# LLMの出力を上記で定義したRecipeモデルに変換するためのパーサー（解析器）を作成
# これにより、LLMから返されたJSON文字列をPythonのRecipeオブジェクトに自動的に変換する
output_parser=PydanticOutputParser(pydantic_object=Recipe)

#----------------------------------------------
# LangChainのプロンプトテンプレートとOpenAIのチャットモデル
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI

# LLMへの指示（プロンプト）の定義
# from_messagesを使うことで、システムメッセージとユーザーメッセージを明確に区別して設定
prompt=ChatPromptTemplate.from_messages(
	[
		(
			# システムの役割を定義。LLMに対する全体的な指示
			"system",
			# ユーザーが入力した料理のレシピを考えてもらうための指示。
			"ユーザーが入力した料理のレシピを考えてください。\n\n"
			# {format_instructions} は、PydanticOutputParserから生成されるJSONフォーマットの指示が挿入されるプレースホルダー。
			"{format_instructions}"
		),
		# ユーザーの役割を定義します。ユーザーが「dish」（料理名）を入力。
		("human","{dish}")
	]
)

# フォーマットの指示をプロンプトに組み込みます。
# prompt.partial()を使うことで、プロンプトの一部（format_instructions）を事前に設定し、
# 後で残りの変数（dish）を入力できるようにします。
# これにより、LLMはPydanticモデルの形式に沿ったJSONを出力しようとします。
prompt_with_format_instructions=prompt.partial(
	format_instructions=output_parser.get_format_instructions()
)

# OpenAIのチャットモデル（gpt-4o-mini）を初期化します。
# temperature=0 は、モデルの出力のランダム性（創造性）を最小限に抑え、より一貫した結果を得るためです。
# .bind(response_format={"type":"json_object"}) は、LLMが必ずJSON形式で応答するように強制する設定です。
# これにより、PydanticOutputParserが正しく動作するためのJSON出力が保証されます。
model=ChatOpenAI(model="gpt-4o-mini",temperature=0).bind(
	response_format={"type":"json_object"}
)

#----------------------------------------------
# プロンプト、モデル、出力パーサーをパイプ（|）で連鎖させ、一つの実行可能なチェーンを構築します。
# このチェーンは、以下の順序で処理を実行します。
# 1. prompt_with_format_instructions: ユーザー入力（料理名）を受け取り、完全なプロンプトを作成します。
# 2. model: 作成されたプロンプトをLLMに渡し、JSON形式の応答を得ます。
# 3. output_parser: LLMからのJSON応答をPythonのRecipeオブジェクトに変換します。
chain=prompt_with_format_instructions|model|output_parser

#----------------------------------------------
# 構築したチェーンを実行し、「カレー」のレシピを生成します。
# chain.invoke()に辞書形式で入力を渡します。
recipe=chain.invoke({"dish":"カレー"})
# 生成されたレシピの型を表示します。これにより、LLMの出力が正しくRecipeクラスのインスタンスに変換されていることを確認できます。
print(type(recipe))
# 生成されたレシピの内容（材料と手順）を表示します。
print(recipe)

<class '__main__.Recipe'>
ingredients=['鶏肉 500g', '玉ねぎ 2個', 'にんじん 1本', 'じゃがいも 2個', 'カレールー 1箱', '水 800ml', 'サラダ油 大さじ2', '塩 適量', 'こしょう 適量'] steps=['鶏肉は一口大に切り、塩とこしょうをふる。', '玉ねぎは薄切り、にんじんは輪切り、じゃがいもは一口大に切る。', '鍋にサラダ油を熱し、玉ねぎを炒めて透明になるまで炒める。', '鶏肉を加え、表面が白くなるまで炒める。', 'にんじんとじゃがいもを加え、さらに炒める。', '水を加え、煮立ったらアクを取り除く。', '弱火にして、約20分煮込む。', 'カレールーを加え、溶かしながらさらに10分煮込む。', '味を見て、必要に応じて塩で調整する。', 'ご飯と一緒に盛り付けて完成。']


### （コラム）with_structured_output


In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from pydantic import BaseModel, Field


class Recipe(BaseModel):
    ingredients: list[str] = Field(description="ingredients of the dish")
    steps: list[str] = Field(description="steps to make the dish")


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "ユーザーが入力した料理のレシピを考えてください。"),
        ("human", "{dish}"),
    ]
)

model = ChatOpenAI(model="gpt-4o-mini")

chain = prompt | model.with_structured_output(Recipe)

recipe = chain.invoke({"dish": "カレー"})
print(type(recipe))
print(recipe)

## 4.6.LangChain の RAG に関するコンポーネント


### Document loader


In [56]:
# GitLoaderをインポート
from langchain_community.document_loaders import GitLoader

# .mdxファイルのみを対象とするフィルタ関数を定義
def file_filter(file_path: str) -> bool:
    # ファイルパスが".mdx"で終わる場合のみTrueを返す
    return file_path.endswith(".mdx")

# GitリポジトリからドキュメントをロードするためのGitLoaderインスタンスを作成
loader = GitLoader(
    clone_url="https://github.com/langchain-ai/langchain",  # クローンするリポジトリのURL
    repo_path="./langchain",                                # ローカルにクローンするパス
    branch="master",                                        # 使用するブランチ
    file_filter=file_filter,                                # 上で定義したファイルフィルタを適用
)

# Gitリポジトリからドキュメントをロードし、raw_docsに格納
raw_docs = loader.load()

# ロードしたドキュメント数を出力
print(len(raw_docs))

418


### Document transformer


In [ ]:
# CharacterTextSplitterクラスをlangchain_text_splittersからインポート
from langchain_text_splitters import CharacterTextSplitter

# チャンクサイズ1000、重なり0でテキスト分割器を作成
text_splitter = CharacterTextSplitter(chunk_size=1000, chunk_overlap=0)

# raw_docsをチャンクごとに分割し、docsに格納
docs = text_splitter.split_documents(raw_docs)

# 分割後のドキュメント数を出力
print(len(docs))

### Embedding model


In [59]:
from langchain_openai import OpenAIEmbeddings

# "text-embedding-3-small"モデルで埋め込みインスタンスを作成
embeddings = OpenAIEmbeddings(model="text-embedding-3-small")

#-----------------------------------------------------------------------
# 質問文を定義
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"
# 質問文をベクトル化（埋め込みを取得）
vector = embeddings.embed_query(query)
# ベクトルの中身を表示
print(len(vector))
print(vector)

1536
[0.02016349323093891, -0.008064329624176025, 0.03321799263358116, -0.027838215231895447, 0.04628317058086395, 0.025383159518241882, -0.01679045706987381, 0.018508998677134514, 0.026877541095018387, -0.02182866260409355, 0.010156465694308281, -0.01113315112888813, -0.015445513650774956, -0.00540913175791502, -0.011752253398299217, 0.06246519833803177, 0.03584383800625801, -0.00037059359601698816, -0.04645395651459694, 0.02589551918208599, 0.030869677662849426, 0.034562937915325165, -0.039110131561756134, 0.02166854962706566, 0.01734551414847374, -0.018594391644001007, -0.020238211378455162, 0.029973048716783524, -0.008726128377020359, -0.10298432409763336, -0.008181745186448097, -0.057384297251701355, -0.034349456429481506, 0.04722249507904053, -0.021946078166365623, 0.035117994993925095, 0.022074168547987938, 0.011464050970971584, -0.007562644314020872, -0.032641589641571045, 0.000674807233735919, -0.021433718502521515, 0.020964056253433228, 0.012403377331793308, 0.006996913813054

### Vector store


In [67]:
from langchain_chroma import Chroma

# ドキュメントと埋め込みモデルからChromaベクトルストアを作成
db = Chroma.from_documents(docs, embeddings)

# ベクトルストアからレトリバー（検索用インターフェース）を作成
retriever = db.as_retriever()

# 質問文を定義
query = "AWSのS3からデータを読み込むためのDocument loaderはありますか？"

# レトリバーを使って関連ドキュメントを検索
context_docs = retriever.invoke(query)
# 検索結果のドキュメント数を表示
print(f"len={len(context_docs)}")

# 最初の検索結果ドキュメントを取得
first_doc = context_docs[0]
# 最初のドキュメントのメタデータを表示
print(f"metadata={first_doc.metadata}")
# 最初のドキュメントの本文を表示
print(first_doc.page_content)


Failed to send telemetry event ClientStartEvent: capture() takes 1 positional argument but 3 were given
Failed to send telemetry event ClientCreateCollectionEvent: capture() takes 1 positional argument but 3 were given


len=4
metadata={'file_name': 'aws.mdx', 'file_path': 'docs/docs/integrations/providers/aws.mdx', 'file_type': '.mdx', 'source': 'docs/docs/integrations/providers/aws.mdx'}
### AWS S3 Directory and File

>[Amazon Simple Storage Service (Amazon S3)](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
> is an object storage service.
>[AWS S3 Directory](https://docs.aws.amazon.com/AmazonS3/latest/userguide/using-folders.html)
>[AWS S3 Buckets](https://docs.aws.amazon.com/AmazonS3/latest/userguide/UsingBucket.html)

See a [usage example for S3DirectoryLoader](/docs/integrations/document_loaders/aws_s3_directory).

See a [usage example for S3FileLoader](/docs/integrations/document_loaders/aws_s3_file).

```python
from langchain_community.document_loaders import S3DirectoryLoader, S3FileLoader
```

### Amazon Textract

>[Amazon Textract](https://docs.aws.amazon.com/managedservices/latest/userguide/textract.html) is a machine 
> learning (ML) service that automatically ex

### LCEL を使った RAG の Chain の実装


In [68]:
# ChatPromptTemplateをインポート（プロンプトテンプレート作成用）
from langchain_core.prompts import ChatPromptTemplate
# ChatOpenAIをインポート（OpenAIのチャットモデル利用用）
from langchain_openai import ChatOpenAI

# プロンプトテンプレートを作成（文脈と質問を埋め込む）
prompt = ChatPromptTemplate.from_template('''\
以下の文脈だけを踏まえて質問に回答してください。

文脈"""
{context}
"""

質問:{question}
''')

# OpenAIのチャットモデル（gpt-4o-mini）を初期化（温度0で出力の一貫性を高める）
model = ChatOpenAI(model_name="gpt-4o-mini", temperature=0)

# ----------------------------------------------------------
# StrOutputParserをインポート（モデル出力を文字列としてパースするため）
from langchain_core.output_parsers import StrOutputParser
# RunnablePassthroughをインポート（入力値をそのまま渡すため）
from langchain_core.runnables import RunnablePassthrough

# チェーンを構築（retrieverで文脈取得→プロンプト→モデル→出力パース）
chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | model
    | StrOutputParser()
)

# チェーンにクエリを渡して実行し、結果を取得
output = chain.invoke(query)
# 結果を表示
print(output)

はい、AWSのS3からデータを読み込むためのDocument loaderとして、`S3DirectoryLoader`と`S3FileLoader`があります。これらは、AWS S3のディレクトリやファイルからデータを読み込むために使用されます。
